In [7]:
import os
import pandas as pd
import json

frame = pd.read_pickle('../frame_all.json')
frame

,sessionId,contentUrl,device,deviceInfo.deviceId,deviceInfo.model,deviceInfo.os,deviceInfo.osVersion,deviceInfo.player,errorCode,event,...,fragmentSum.duration,fragmentSum.fragmentIndex,fragmentSum.size,fragments,frameDropped,maxDecodingTime,networkChanged,networkErrors,qualityChanged,userEvents
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# test = pd.read_json('raw_data/06/02/qoe.dtv-12-2018-11-06-02-14-13-e14e6911-fb2e-4876-b2d6-8ffef3b07938', lines=True)
# test = pd.io.json.json_normalize(test.qoe)
# set(test.event)

In [18]:
def remove_and_merge(file_path):
    rdf = pd.read_json(file_path, lines=True)
    qoe = pd.io.json.json_normalize(rdf.qoe)
    frame = pd.read_pickle('all.json')    
    try:
        qoe_ended = qoe.drop(qoe[qoe.event!='PlaybackEnded'].index)
        qoe_updated = qoe.drop(qoe[qoe.event!='PlaybackUpdated'].index)

        qoe_ended.dropna(axis=1, how='all', thresh=None, subset=None, inplace=True)
        qoe_updated.dropna(axis=1, how='all', thresh=None, subset=None, inplace=True)

        qoe_updated.sort_values(by=['sessionId', 'timestamp'], inplace=True)
        qoe_updated.drop_duplicates(subset=['sessionId'], keep='last', inplace=True)

        qoe_ended.index = qoe_ended['sessionId']
        qoe_updated.index = qoe_updated['sessionId']

        qoe_updated = qoe_updated.drop(columns=['contentUrl', 'device', 'deviceInfo.deviceId', 'deviceInfo.model',
           'deviceInfo.os', 'deviceInfo.osVersion', 'deviceInfo.player', 'event',
           'licenseInfo.drmSystem', 'licenseInfo.elapsedTime', 'licenseInfo.laUrl',
           'networkInfo.carrier.mcc', 'networkInfo.carrier.mnc',
           'networkInfo.carrier.name', 'networkInfo.type', 'qoeContentId',
           'redirectUrl', 'sessionId', 'timestamp'], axis = 1)

        merged_version =qoe_ended.join(qoe_updated, on=qoe_ended.index, how='left')
        merged_version = merged_version.drop(columns= ['sessionId'], axis = 1)
        merged_version.reset_index(inplace=True)
        
        merged_version.to_pickle(file_path)
        
        qoe = pd.read_pickle(file_path)
        frame = pd.concat([frame, qoe])

    except:
        os.remove(file_path)
        pass
    
    frame.to_pickle('all.json')

In [19]:
raw_data_dir = "/home/centos/raw_data"
for day_dir in os.listdir(raw_data_dir):
    day_dir = os.path.join(raw_data_dir, day_dir)
    for hour_dir in os.listdir(day_dir):
        hour_dir = os.path.join(day_dir, hour_dir)
        for file_path in os.listdir(hour_dir):
            file_path = os.path.join(hour_dir, file_path)
            print(file_path)
            remove_and_merge(file_path)

/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-04-34-b5eb4b72-807e-4f45-9c94-514a8c706e1f
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-05-05-a3fdfbb8-6786-4ab2-a9ef-35307b8d5378
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-08-47-94574f17-b6fe-4682-93fa-8a81d33627ff
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-08-16-2a21eabd-0b0a-41da-846d-b8aab9d3e6e5
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-10-35-208ca486-77e9-40eb-a409-fe3d04fb2820
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-06-54-278e5bfe-fffc-4559-a914-0ff86a135244
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-00-38-12986c32-8d18-4438-a613-2eeee48dbd0b
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-19-57-2e9b0c6b-2446-447e-9ec7-7a252c88b994
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-00-21-4d3849d0-ae23-4f82-904a-af6b1a97f8de
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-01-28-2faf6c28-bad4-43b8-b62d-8e2fadc3a92a
/home/centos/raw_data/14/02/qoe.dtv-12-2

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-06-21-94d99238-0d1e-4275-b552-88c7646b9eee
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-07-27-ab685d39-5b91-459b-9c51-0ee9df47b04f


/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-09-35-19e1f906-2e23-4dc8-bebc-b3de91b4067e
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-02-53-f099d1b0-0097-42cb-8787-9961f5710409


...

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-59-43-26296640-2309-4586-b029-ce0dd17fb16c
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-59-56-9f0ff3ee-d86a-439b-ac95-71cdb6d3270d


/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-59-32-debab402-481b-4479-b1b7-b5b063f778d5
/home/centos/raw_data/14/02/qoe.dtv-12-2018-11-14-02-59-05-dfd37df4-4bfd-40ee-859f-018f82850946


/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [ ]:
# remove_and_merge('raw_data/06/02/qoe.dtv-12-2018-11-06-02-14-13-e14e6911-fb2e-4876-b2d6-8ffef3b07938')

In [2]:
import pandas as pd
all_qoe = pd.read_pickle('../all.json')
all_qoe

EOFError: 